In [1]:
import xarray as xr
import numpy as np
import dask.array as da
import matplotlib.pyplot as plt
import os
from dask.distributed import Client, LocalCluster
from datetime import datetime,timedelta
import glob
import rioxarray
import xarray as xr
from glob import glob
import rioxarray as rio
import cftime
import indices_function as ifun


In [ ]:
cluster = LocalCluster(
    n_workers=40, 
    threads_per_worker=1,
    timeout='3600s',
    memory_limit='20GB',   
)
client = Client(cluster)
client

In [13]:
hi_precip=xr.open_mfdataset("/nobackupp17/bthrashe/public/HI_test/tasmin*historical*.nc",decode_times=False)


In [25]:
!ls /nobackupp27/nex/datapool/reanalysis/hawaii/rainfall/new/day

statewide


In [26]:
# Define the path to your TIF files
tif_files = sorted(glob("/nobackupp27/nex/datapool/reanalysis/hawaii/rainfall/new/day/statewide/data_map/*/*/*.tif"))
# Open each TIF file as a DataArray
tif_files = [file for file in tif_files if "02_29" not in file]
data_arrays = [rioxarray.open_rasterio(tif_file, chunks={'x': 1024, 'y': 1024}) for tif_file in tif_files[:9125]]

# Concatenate along a new dimension (e.g., 'time' or 'band')
combined_data = xr.concat(data_arrays, dim="time")
combined_data = combined_data.rename({"y": "lat", "x": "lon"})

# Squeeze out the 'band' dimension if it has only one value
if 'band' in combined_data.dims:
    combined_data = combined_data.squeeze("band", drop=True)
combined_data = combined_data.assign_coords(time=("time", times[14600:]))
combined_data.to_zarr("/nobackupp28/skhajehe/hawaii_rea/hawaii_reanalysis_pr.zarr")

/home5/skhajehe/.local/lib/python3.10/site-packages/distributed/client.py:3169: UserWarning: Sending large graph of size 19.72 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [ ]:
combined_data.to_zarr("/nobackupp28/skhajehe/hawaii_rea/hawaii_reanalysis_min.zarr")

In [ ]:
import xarray as xr
reanalysis=xr.open_zarr("/nobackupp28/skhajehe/hawaii_rea/hawaii_reanalysis_max.zarr")
reanalysis = reanalysis.rename({"__xarray_dataarray_variable__": "max_temp"})
max_temp=reanalysis.chunk(dict(time=-1,lat=200,lon=200)).groupby('time.season').mean(dim='time')
max_temp.to_zarr("/nobackupp28/skhajehe/hawaii_rea/hawaii_reanalysis_max_seasonal.zarr",mode="w")


In [29]:
# import xarray as xr
reanalysis=xr.open_zarr("/nobackupp28/skhajehe/hawaii_rea/hawaii_reanalysis_max.zarr")
reanalysis = reanalysis.rename({"__xarray_dataarray_variable__": "tasmax"})
pr=reanalysis.chunk(dict(time=-1,lat=200,lon=200)).mean(dim='time')
pr.to_zarr("/nobackupp28/skhajehe/hawaii_rea/hawaii_reanalysis_tasmax_annual.zarr",mode="w")


In [42]:
ls /nobackupp28/skhajehe/hawaii_rea/

hawaii_gcm_max_annual.zarr/    hawaii_reanalysis_max_annual.zarr/
hawaii_gcm_max_quantile.zarr/  hawaii_reanalysis_max_quantile.zarr/
hawaii_gcm_max_seasonal.zarr/  hawaii_reanalysis_max_seasonal.zarr/
hawaii_gcm_max.zarr/           hawaii_reanalysis_max.zarr/
hawaii_gcm_min_annual.zarr/    hawaii_reanalysis_min_annual.zarr/
hawaii_gcm_min_quantile.zarr/  hawaii_reanalysis_min_quantile.zarr/
hawaii_gcm_min_seasonal.zarr/  hawaii_reanalysis_min_seasonal.zarr/
hawaii_gcm_min.zarr/           hawaii_reanalysis_min.zarr/
hawaii_gcm_pr_annual.zarr/     hawaii_reanalysis_pr_annual.zarr/
hawaii_gcm_pr_quantile.zarr/   hawaii_reanalysis_pr_quantile.zarr/
hawaii_gcm_pr.zarr/            hawaii_reanalysis.zarr/


In [ ]:
xr.open_zarr("/nobackupp28/skhajehe/hawaii_rea/hawaii_reanalysis_min.zarr")

In [32]:
reanalysis=xr.open_zarr("/nobackupp28/skhajehe/hawaii_rea/hawaii_reanalysis_pr.zarr")
reanalysis = reanalysis.rename({"__xarray_dataarray_variable__": "pr"})
quantile_models = ifun.nonzero_quantiles(reanalysis.chunk(dict(time=-1,lat=200,lon=200)),[0.25,0.5,0.75,0.9])
quantile_models.to_zarr("/nobackupp28/skhajehe/hawaii_rea/hawaii_reanalysis_pr_quantile.zarr",mode="w")


In [38]:
reanalysis=xr.open_zarr("/nobackupp28/skhajehe/hawaii_rea/hawaii_gcm_min.zarr")
quantile_models = ifun.nonzero_quantiles(reanalysis.chunk(dict(time=-1,lat=200,lon=200)),[0.25,0.5,0.75,0.1])
quantile_models.to_zarr("/nobackupp28/skhajehe/hawaii_rea/hawaii_gcm_min_quantile.zarr",mode="w")


2024-10-26 12:22:02,980 - distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)


In [23]:
hi_precip=xr.open_mfdataset("/nobackupp17/bthrashe/public/HI_test/tasmin*historical*.nc",decode_times=False)
times = cftime.num2date(hi_precip['time'].values, units=hi_precip['time'].units, calendar=hi_precip['time'].calendar)
hi_precip['time'] = times
hi_precip.sel(time=slice('1990','2014')).to_zarr("/nobackupp28/skhajehe/hawaii_rea/hawaii_gcm_min.zarr")

In [25]:
reanalysis=xr.open_zarr("/nobackupp28/skhajehe/hawaii_rea/hawaii_gcm_max.zarr")
max_temp=reanalysis.chunk(dict(time=-1,lat=200,lon=200)).groupby('time.season').mean(dim='time')
max_temp.to_zarr("/nobackupp28/skhajehe/hawaii_rea/hawaii_gcm_max_seasonal.zarr",mode="w")

In [40]:
# import xarray as xr
reanalysis=xr.open_zarr("/nobackupp28/skhajehe/hawaii_rea/hawaii_gcm_pr.zarr")
pr=reanalysis.chunk(dict(time=-1,lat=200,lon=200)).mean(dim='time')
pr.to_zarr("/nobackupp28/skhajehe/hawaii_rea/hawaii_gcm_pr_annual.zarr",mode="w")


In [20]:
!ls /nobackupp27/nex/projects/GDDP2/HI/BCSD/ACCESS-CM2/historical/r1i1p1f1/tasmin

tasmin_day_ACCESS-CM2_historical_r1i1p1f1_gn_1950.nc
tasmin_day_ACCESS-CM2_historical_r1i1p1f1_gn_1951.nc
tasmin_day_ACCESS-CM2_historical_r1i1p1f1_gn_1952.nc
tasmin_day_ACCESS-CM2_historical_r1i1p1f1_gn_1953.nc
tasmin_day_ACCESS-CM2_historical_r1i1p1f1_gn_1954.nc
tasmin_day_ACCESS-CM2_historical_r1i1p1f1_gn_1955.nc
tasmin_day_ACCESS-CM2_historical_r1i1p1f1_gn_1956.nc
tasmin_day_ACCESS-CM2_historical_r1i1p1f1_gn_1957.nc
tasmin_day_ACCESS-CM2_historical_r1i1p1f1_gn_1958.nc
tasmin_day_ACCESS-CM2_historical_r1i1p1f1_gn_1959.nc
tasmin_day_ACCESS-CM2_historical_r1i1p1f1_gn_1960.nc
tasmin_day_ACCESS-CM2_historical_r1i1p1f1_gn_1961.nc
tasmin_day_ACCESS-CM2_historical_r1i1p1f1_gn_1962.nc
tasmin_day_ACCESS-CM2_historical_r1i1p1f1_gn_1963.nc
tasmin_day_ACCESS-CM2_historical_r1i1p1f1_gn_1964.nc
tasmin_day_ACCESS-CM2_historical_r1i1p1f1_gn_1965.nc
tasmin_day_ACCESS-CM2_historical_r1i1p1f1_gn_1966.nc
tasmin_day_ACCESS-CM2_historical_r1i1p1f1_gn_1967.nc
tasmin_day_ACCESS-CM2_historical_r1i1p1f1_gn_1